In [9]:
# Part 4: Advanced Distribution Risk Models and VaR/ES Analysis
import pandas as pd
import numpy as np
from scipy.special import gamma
from statsmodels.distributions.empirical_distribution import ECDF
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from scipy.optimize import minimize
from scipy.special import kv
from scipy.integrate import quad


# First, read and process the portfolio data
# Step 1: Load the data
daily_prices = pd.read_csv('../Projects/Final Project/DailyPrices.csv')
initial_portfolio = pd.read_csv('../Projects/Final Project/initial_portfolio.csv')
risk_free = pd.read_csv('../Projects/Final Project/rf.csv')

# Convert dates to datetime format
daily_prices['Date'] = pd.to_datetime(daily_prices['Date'])
risk_free['Date'] = pd.to_datetime(risk_free['Date'])

# Step 2: Split data into training (2023) and testing (2024-2025) periods
training_data = daily_prices[daily_prices['Date'] <= pd.Timestamp('2023-12-29')]
testing_data = daily_prices[daily_prices['Date'] >= pd.Timestamp('2023-12-29')]


# Step 3: Calculate returns
# Function to calculate daily returns
def calculate_returns(prices_df):
    returns_df = prices_df.copy()
    for column in prices_df.columns:
        if column != 'Date':
            returns_df[column] = prices_df[column].pct_change()
    return returns_df.dropna()


# Calculate returns for training and testing periods
training_returns = calculate_returns(training_data)
testing_returns = calculate_returns(testing_data)


# # Get unique portfolio names
portfolios = initial_portfolio['Portfolio'].unique().tolist()
print(f"Unique portfolios: {portfolios}")

# Calculate the total value of each portfolio
portfolio_values = {}
for portfolio in portfolios:
    portfolio_holdings = initial_portfolio[initial_portfolio['Portfolio'] == portfolio]
    total_value = portfolio_holdings['Holding'].sum()
    portfolio_values[portfolio] = total_value

# Create the portfolio_weights dictionary with the correct structure
portfolio_weights = {}
for portfolio in portfolios:
    portfolio_weights[portfolio] = {}

    # Get holdings for this portfolio
    portfolio_holdings = initial_portfolio[initial_portfolio['Portfolio'] == portfolio]

    # Calculate total portfolio value
    total_value = portfolio_holdings['Holding'].sum()

    # Calculate weight for each symbol
    for _, row in portfolio_holdings.iterrows():
        symbol = row['Symbol']
        holding = row['Holding']
        weight = holding / total_value if total_value > 0 else 0

        portfolio_weights[portfolio][symbol] = {
            'weight': weight,
            'holding': holding
        }

print("Portfolio weights successfully created.")

class NormalInverseGaussian:
    """
    Custom implementation of the Normal Inverse Gaussian distribution.
    """
    def __init__(self, alpha, beta, mu, delta):
        # Parameter checks
        if alpha <= 0:
            raise ValueError("alpha must be positive")
        if abs(beta) >= alpha:
            raise ValueError("abs(beta) must be less than alpha")
        if delta <= 0:
            raise ValueError("delta must be positive")

        self.alpha = alpha
        self.beta = beta
        self.mu = mu
        self.delta = delta
        # Derived parameter
        self.gamma = np.sqrt(alpha**2 - beta**2)

    def pdf(self, x):
        """Probability density function"""
        alpha, beta, mu, delta = self.alpha, self.beta, self.mu, self.delta
        gamma = self.gamma

        # Handle array input
        if np.isscalar(x):
            x = np.array([x])
        else:
            x = np.asarray(x)

        # Calculate components
        arg = alpha * np.sqrt(delta**2 + (x - mu)**2)

        # Calculate PDF
        pdf_values = (alpha * delta * kv(1, arg) *
                      np.exp(delta * gamma + beta * (x - mu)) /
                      (np.pi * np.sqrt(delta**2 + (x - mu)**2)))

        # Handle potential numerical issues
        pdf_values = np.maximum(pdf_values, 1e-300)

        if len(pdf_values) == 1:
            return pdf_values[0]
        return pdf_values

    def fit(self, data):
        """Fit distribution parameters using MLE"""
        data = np.asarray(data)

        # Define negative log-likelihood function
        def neg_loglikelihood(params):
            alpha, beta, mu, delta = params
            if alpha <= 0 or delta <= 0 or abs(beta) >= alpha:
                return np.inf

            try:
                model = NormalInverseGaussian(alpha, beta, mu, delta)
                pdf_values = model.pdf(data)
                # Handle numerical issues
                pdf_values = np.maximum(pdf_values, 1e-300)
                return -np.sum(np.log(pdf_values))
            except:
                return np.inf

        # Initial parameter estimates
        mean = np.mean(data)
        var = np.var(data)
        skew = stats.skew(data)
        kurtosis = stats.kurtosis(data, fisher=False)

        # Initial estimates
        try:
            if kurtosis > 3:  # Must be leptokurtic for NIG
                delta_init = 3 * var / (kurtosis - 3)
                alpha_init = np.sqrt(3 * kurtosis / (var * (kurtosis - 3)))
                beta_init = skew / (var * np.sqrt(kurtosis - 3)) if skew != 0 else 0
                mu_init = mean - beta_init * delta_init / np.sqrt(alpha_init**2 - beta_init**2)
            else:
                # Fallback if kurtosis doesn't meet requirements
                delta_init = var
                alpha_init = 2.0 / np.sqrt(var)
                beta_init = skew / (2.0 * var) if skew != 0 else 0
                mu_init = mean
        except:
            # Simple fallback if moment-based estimates fail
            delta_init = np.std(data)
            alpha_init = 1.5 / delta_init
            beta_init = 0
            mu_init = mean

        # Ensure alpha > |beta|
        if abs(beta_init) >= alpha_init:
            alpha_init = abs(beta_init) + 0.1

        # Initial parameters
        initial_params = [alpha_init, beta_init, mu_init, delta_init]

        # Optimize
        try:
            result = minimize(neg_loglikelihood, initial_params,
                             method='Nelder-Mead',
                             bounds=[(0.001, None), (None, None), (None, None), (0.001, None)])

            if result.success:
                alpha, beta, mu, delta = result.x
                return alpha, beta, mu, delta
            else:
                return alpha_init, beta_init, mu_init, delta_init
        except:
            return alpha_init, beta_init, mu_init, delta_init

    def cdf(self, x):
        """Cumulative distribution function"""
        if np.isscalar(x):
            lower_bound = x - 50 * self.delta
            result, _ = quad(self.pdf, lower_bound, x)
            return result
        else:
            return np.array([self.cdf(xi) for xi in x])

    def ppf(self, q):
        """Percent point function (inverse CDF)"""
        if np.isscalar(q):
            if q <= 0: return -np.inf
            if q >= 1: return np.inf

            x_min, x_max = self.mu - 50 * self.delta, self.mu + 50 * self.delta

            # Expand range if needed
            attempts = 0
            while attempts < 10:
                if self.cdf(x_min) > q:
                    x_min -= 50 * self.delta
                elif self.cdf(x_max) < q:
                    x_max += 50 * self.delta
                else:
                    break
                attempts += 1

            # Binary search
            for _ in range(50):
                x_mid = (x_min + x_max) / 2
                cdf_mid = self.cdf(x_mid)

                if abs(cdf_mid - q) < 1e-6:
                    return x_mid

                if cdf_mid < q:
                    x_min = x_mid
                else:
                    x_max = x_mid

            return (x_min + x_max) / 2
        else:
            return np.array([self.ppf(qi) for qi in q])


# 2. Fit distributions to pre-holding period data
print("\nFitting distributions to pre-holding period stock returns...")

# Use training data for fitting
fit_results = {}
best_models = {}
model_params = {}
stock_returns = {}

# List of symbols excluding Date
symbols = [col for col in training_returns.columns if col not in ['Date', 'rf']]
symbols = [s for s in symbols if not s.endswith('_excess')]

# Define a function to evaluate model fit using AIC
def calculate_aic(log_likelihood, k):
    return 2 * k - 2 * log_likelihood

# Function to fit all distributions to a stock's returns
def fit_distributions(returns):
    result = {}

    # Filter out any NaN values - Fix for numpy arrays
    if isinstance(returns, np.ndarray):
        clean_returns = returns[~np.isnan(returns)]
    else:
        # If it's a pandas Series or DataFrame
        clean_returns = returns.dropna()

    # 1. Normal distribution
    try:
        norm_params = stats.norm.fit(clean_returns)
        mu, sigma = norm_params
        log_likelihood = np.sum(stats.norm.logpdf(clean_returns, mu, sigma))
        aic = calculate_aic(log_likelihood, 2)  # 2 parameters: mu, sigma
        result['Normal'] = {
            'params': norm_params,
            'aic': aic,
            'dist': stats.norm(*norm_params)
        }
    except:
        result['Normal'] = {'aic': np.inf}

    # 2. Generalized T distribution
    try:
        t_params = stats.t.fit(clean_returns)
        log_likelihood = np.sum(stats.t.logpdf(clean_returns, *t_params))
        aic = calculate_aic(log_likelihood, 3)  # 3 parameters: df, loc, scale
        result['GeneralizedT'] = {
            'params': t_params,
            'aic': aic,
            'dist': stats.t(*t_params)
        }
    except Exception as e:
        print(f"Error fitting GeneralizedT: {e}")
        result['GeneralizedT'] = {'aic': np.inf}

    # 3. NIG distribution
    try:
        nig = NormalInverseGaussian(1, 0, 0, 1)  # Default initialization
        alpha, beta, mu, delta = nig.fit(clean_returns)
        nig_params = (alpha, beta, mu, delta)
        nig_fitted = NormalInverseGaussian(*nig_params)

        # Calculate log-likelihood and AIC
        pdf_values = nig_fitted.pdf(clean_returns)
        pdf_values = np.maximum(pdf_values, 1e-300)  # Avoid log(0)
        log_likelihood = np.sum(np.log(pdf_values))
        aic = calculate_aic(log_likelihood, 4)  # 4 parameters: alpha, beta, mu, delta

        result['NIG'] = {
            'params': nig_params,
            'aic': aic,
            'dist': nig_fitted
        }
    except Exception as e:
        print(f"Error fitting NIG: {e}")
        import traceback
        traceback.print_exc()  # Print the full traceback for debugging
        result['NIG'] = {'aic': np.inf}


# 4. Skew Normal
    try:
        skewnorm_params = stats.skewnorm.fit(clean_returns)
        log_likelihood = np.sum(stats.skewnorm.logpdf(clean_returns, *skewnorm_params))
        aic = calculate_aic(log_likelihood, 3)  # 3 parameters: a, loc, scale
        result['SkewNormal'] = {
            'params': skewnorm_params,
            'aic': aic,
            'dist': stats.skewnorm(*skewnorm_params)
        }
    except Exception as e:
        print(f"Error fitting SkewNormal: {e}")
        result['SkewNormal'] = {'aic': np.inf}

    # Find best model based on AIC
    best_model = min(result.items(), key=lambda x: x[1]['aic'])[0]

    return result, best_model

# Fit distributions for each stock
for symbol in symbols:
    # Extract training period returns
    returns = training_returns[symbol].values
    stock_returns[symbol] = returns

    # Fit all distributions and find the best one
    try:
        fit_results[symbol], best_models[symbol] = fit_distributions(returns)
        model_params[symbol] = fit_results[symbol][best_models[symbol]]['params']
        # print(f"{symbol}: Best fit model is {best_models[symbol]}")
    except Exception as e:
        print(f"Error fitting distributions for {symbol}: {e}")
        best_models[symbol] = "Normal"  # Default to normal if fitting fails
        model_params[symbol] = stats.norm.fit(returns)

# Report best fit models and parameters
print("\nBest Fit Distribution Models and Parameters:")
print("=" * 100)
print(f"{'Symbol':<8} {'Best Model':<15} {'Parameters'}")
print("-" * 100)

for symbol in symbols:
    params = model_params[symbol]
    rounded_params = tuple([round(float(p), 8) for p in params])
    print(f"{symbol:<8} {best_models[symbol]:<15} {rounded_params}")

# 3. Calculate VaR and ES using Gaussian Copula with fitted marginals
print("\nCalculating VaR and ES using Gaussian Copula with fitted marginals...")

# Function to calculate portfolio VaR and ES
def calculate_var_es(portfolio_name, weights, confidence_level=0.95, n_simulations=10000, method="GaussianCopula"):
    symbols_in_portfolio = list(weights.keys())

    if method == "GaussianCopula":
        # Step 1: Transform original returns to uniform using fitted distributions
        uniform_data = {}
        for symbol in symbols_in_portfolio:
            returns = stock_returns[symbol]
            best_model = best_models[symbol]
            dist = fit_results[symbol][best_model]['dist']

            # Calculate empirical CDFs
            try:
                u = np.array([dist.cdf(x) for x in returns])
                # Handle boundary cases
                u = np.minimum(np.maximum(u, 0.0001), 0.9999)
                uniform_data[symbol] = u
            except Exception as e:
                print(f"Error transforming {symbol} to uniform: {e}")
                # Fallback to empirical CDF
                ecdf = ECDF(returns)
                u = ecdf(returns)
                uniform_data[symbol] = u

        # Step 2: Transform uniform to standard normal
        normal_data = {}
        for symbol in symbols_in_portfolio:
            try:
                normal_data[symbol] = stats.norm.ppf(uniform_data[symbol])
            except:
                # Handle any numerical issues
                u_clean = np.clip(uniform_data[symbol], 0.0001, 0.9999)
                normal_data[symbol] = stats.norm.ppf(u_clean)

        # Step 3: Estimate correlation matrix of transformed data
        transformed_returns = pd.DataFrame({symbol: normal_data[symbol] for symbol in symbols_in_portfolio})
        correlation_matrix = transformed_returns.corr().values

        # Step 4: Generate correlated normal samples
        np.random.seed(42)  # For reproducibility
        simulated_normals = np.random.multivariate_normal(
            mean=np.zeros(len(symbols_in_portfolio)),
            cov=correlation_matrix,
            size=n_simulations
        )

        # Step 5: Transform back to original distribution
        simulated_returns = np.zeros((n_simulations, len(symbols_in_portfolio)))

        for i, symbol in enumerate(symbols_in_portfolio):
            z = simulated_normals[:, i]
            u = stats.norm.cdf(z)

            # Get correct distribution
            best_model = best_models[symbol]
            dist = fit_results[symbol][best_model]['dist']

            # Transform uniform back to returns using inverse CDF (ppf)
            try:
                simulated_returns[:, i] = dist.ppf(u)
            except Exception as e:
                print(f"Error in inverse transform for {symbol}: {e}")
                # Fallback to empirical inverse CDF
                x_sorted = np.sort(stock_returns[symbol])
                indices = np.floor(u * len(x_sorted)).astype(int)
                indices = np.minimum(indices, len(x_sorted)-1)
                simulated_returns[:, i] = x_sorted[indices]

        # Step 6: Calculate portfolio returns
        portfolio_returns = np.zeros(n_simulations)
        for i, symbol in enumerate(symbols_in_portfolio):
            portfolio_returns += simulated_returns[:, i] * weights[symbol]

    elif method == "MultivariateNormal":
        # Simpler approach: assume multivariate normal directly
        returns_data = np.column_stack([stock_returns[symbol] for symbol in symbols_in_portfolio])

        # Estimate mean and covariance
        mean_vector = np.zeros(len(symbols_in_portfolio))  # Assume 0% return as specified
        cov_matrix = np.cov(returns_data, rowvar=False)

        # Generate multivariate normal samples
        np.random.seed(42)  # For reproducibility
        simulated_returns = np.random.multivariate_normal(
            mean=mean_vector,
            cov=cov_matrix,
            size=n_simulations
        )

        # Calculate portfolio returns
        portfolio_returns = np.zeros(n_simulations)
        for i, symbol in enumerate(symbols_in_portfolio):
            portfolio_returns += simulated_returns[:, i] * weights[symbol]

    # Calculate VaR and ES
    sorted_returns = np.sort(portfolio_returns)
    var_index = int(n_simulations * (1 - confidence_level))
    var = -sorted_returns[var_index]
    es = -np.mean(sorted_returns[:var_index])

    return var, es

# Extract portfolio weights from original structure
# Note: We're using a different variable name to avoid conflict
portfolio_weight_data = {}
for portfolio in portfolios:
    weights = {}
    for symbol, info in portfolio_weights[portfolio].items():
        weights[symbol] = info['weight']
    portfolio_weight_data[portfolio] = weights

# Calculate VaR and ES for each portfolio using both methods
var_es_results = {}
confidence_level = 0.95  # 95% confidence level

for portfolio in portfolios:
    weights = portfolio_weight_data[portfolio]

    # Calculate using Gaussian Copula with fitted marginals
    var_gc, es_gc = calculate_var_es(
        portfolio, weights,
        confidence_level=confidence_level,
        method="GaussianCopula"
    )

    # Calculate using Multivariate Normal
    var_mvn, es_mvn = calculate_var_es(
        portfolio, weights,
        confidence_level=confidence_level,
        method="MultivariateNormal"
    )

    var_es_results[portfolio] = {
        'GaussianCopula': {'VaR': var_gc, 'ES': es_gc},
        'MultivariateNormal': {'VaR': var_mvn, 'ES': es_mvn}
    }

# Calculate for combined portfolio
combined_weights = {}
for portfolio in portfolios:
    portfolio_value = portfolio_values[portfolio]
    for symbol, info in portfolio_weights[portfolio].items():
        weight = info['weight'] * portfolio_value / sum(portfolio_values.values())
        if symbol in combined_weights:
            combined_weights[symbol] += weight
        else:
            combined_weights[symbol] = weight

# Calculate VaR and ES for combined portfolio
var_gc, es_gc = calculate_var_es(
    'Combined', combined_weights,
    confidence_level=confidence_level,
    method="GaussianCopula"
)

var_mvn, es_mvn = calculate_var_es(
    'Combined', combined_weights,
    confidence_level=confidence_level,
    method="MultivariateNormal"
)

var_es_results['Combined'] = {
    'GaussianCopula': {'VaR': var_gc, 'ES': es_gc},
    'MultivariateNormal': {'VaR': var_mvn, 'ES': es_mvn}
}

# Report VaR and ES results
print("\n1-Day VaR and ES Results at 95% Confidence Level:")
print("=" * 100)
print(f"{'Portfolio':<10} {'VaR (GC)':<15} {'ES (GC)':<15} {'VaR (MVN)':<15} {'ES (MVN)':<15} {'VaR Diff %':<15} {'ES Diff %':<15}")
print("-" * 100)

for portfolio in var_es_results:
    var_gc = var_es_results[portfolio]['GaussianCopula']['VaR']
    es_gc = var_es_results[portfolio]['GaussianCopula']['ES']
    var_mvn = var_es_results[portfolio]['MultivariateNormal']['VaR']
    es_mvn = var_es_results[portfolio]['MultivariateNormal']['ES']

    # Calculate percentage differences
    var_diff_pct = (var_gc - var_mvn) / var_mvn * 100 if var_mvn != 0 else float('inf')
    es_diff_pct = (es_gc - es_mvn) / es_mvn * 100 if es_mvn != 0 else float('inf')

    print(f"{portfolio:<10} {var_gc:<15.6f}  {es_gc:<15.6f}  {var_mvn:<15.6f} {es_mvn:<15.6f}  {var_diff_pct:<15.4f} {es_diff_pct:<15.4f}")

print("\nNote: GC = Gaussian Copula with fitted marginals, MVN = Multivariate Normal")
print("      Positive difference percentages indicate that the Gaussian Copula method gives higher risk estimates")

# Additional analysis of the differences
print("\nComparison of Distribution Models vs. Normal Distribution:")
print("=" * 100)
print("The differences between the two approaches can be attributed to:")
print("1. Skewness and fat tails captured by specialized distributions")
print("2. Non-linear dependency structures captured by the copula approach")
print("3. Different assumptions about the joint distribution of returns")

# Analyze which stocks deviate most from normality
print("\nStocks Deviating Most from Normality:")
print("=" * 80)
print(f"{'Symbol':<8} {'Best Model':<15} {'Skewness':<12} {'Excess Kurtosis':<15}")
print("-" * 80)

for symbol in symbols:
    returns = stock_returns[symbol]
    skewness = stats.skew(returns)
    kurtosis = stats.kurtosis(returns)  # Excess kurtosis (normal = 0)

    print(f"{symbol:<8} {best_models[symbol]:<15} {skewness:<12.4f} {kurtosis:<15.4f}")



from collections import Counter

distribution_counts = Counter(best_models.values())

print("\n分布类型对应的股票数量统计：")
print("=" * 40)
for dist, count in distribution_counts.items():
    print(f"{dist:<15} : {count} stocks")

Unique portfolios: ['A', 'B', 'C']
Portfolio weights successfully created.

Fitting distributions to pre-holding period stock returns...

Best Fit Distribution Models and Parameters:
Symbol   Best Model      Parameters
----------------------------------------------------------------------------------------------------
SPY      Normal          (0.000985, 0.00823027)
AAPL     GeneralizedT    (7.32705184, 0.00176004, 0.01070133)
NVDA     GeneralizedT    (4.78942593, 0.0035959, 0.02167961)
MSFT     GeneralizedT    (7.77186949, 0.0016953, 0.01361571)
AMZN     GeneralizedT    (5.92185868, 0.00217428, 0.0168987)
META     GeneralizedT    (4.21960376, 0.00276747, 0.01569186)
GOOGL    GeneralizedT    (4.42004258, 0.0017383, 0.01418544)
AVGO     GeneralizedT    (4.30167917, 0.00138822, 0.01470093)
TSLA     GeneralizedT    (6.49117953, 0.00318664, 0.02780702)
GOOG     GeneralizedT    (4.59360274, 0.00182357, 0.01447821)
BRK-B    GeneralizedT    (6.74524696, 0.00067377, 0.00724056)
JPM      General

In [10]:
# Improved Risk Parity Portfolio Optimization using ES
import numpy as np
from scipy.optimize import minimize
import time

# Define a much more efficient function to calculate marginal risk contributions
def calculate_marginal_risk_contributions(weights, portfolio_name, confidence_level=0.95, n_simulations=500):
    """
    Calculate marginal risk contributions to ES for each asset in the portfolio more efficiently.

    Args:
        weights: Dictionary with asset weights
        portfolio_name: Name of the portfolio
        confidence_level: Confidence level for ES calculation
        n_simulations: Number of simulations for Monte Carlo

    Returns:
        Dictionary of marginal risk contributions indexed by symbol
    """
    symbols_in_portfolio = list(weights.keys())

    # Convert dictionary to numpy array for faster computation
    weights_array = np.array([weights[symbol] for symbol in symbols_in_portfolio])

    # Create returns matrix for faster computation
    returns_matrix = np.zeros((len(stock_returns[symbols_in_portfolio[0]]), len(symbols_in_portfolio)))
    for i, symbol in enumerate(symbols_in_portfolio):
        returns_matrix[:, i] = stock_returns[symbol]

    # Calculate covariance matrix for approximating initial marginal contributions
    cov_matrix = np.cov(returns_matrix, rowvar=False)

    # Approximate marginal contributions using covariance matrix
    # This gives a reasonable starting point while being much faster
    portfolio_variance = weights_array.T @ cov_matrix @ weights_array
    marginal_contrib_approx = (cov_matrix @ weights_array) / np.sqrt(portfolio_variance)

    # For small simulations, use the covariance approximation
    if n_simulations < 300:
        # Create result dictionary
        mrc = {}
        for i, symbol in enumerate(symbols_in_portfolio):
            mrc[symbol] = marginal_contrib_approx[i]
        return mrc

    # For portfolios, use a more accurate but still efficient simulation approach
    try:
        # Use Gaussian Copula method with minimal simulations
        np.random.seed(42)

        # Transform original returns to standard normal
        transformed_returns = np.zeros_like(returns_matrix)
        for i, symbol in enumerate(symbols_in_portfolio):
            # Use empirical CDF for speed
            ecdf = ECDF(returns_matrix[:, i])
            u = ecdf(returns_matrix[:, i])
            u = np.clip(u, 0.001, 0.999)  # Avoid boundary issues
            transformed_returns[:, i] = stats.norm.ppf(u)

        # Calculate correlation matrix (faster than full copula transformation)
        corr_matrix = np.corrcoef(transformed_returns, rowvar=False)

        # Generate correlated normal samples
        simulated_normals = np.random.multivariate_normal(
            mean=np.zeros(len(symbols_in_portfolio)),
            cov=corr_matrix,
            size=n_simulations
        )

        # Transform back to return space using simple method
        simulated_returns = np.zeros((n_simulations, len(symbols_in_portfolio)))
        for i, symbol in enumerate(symbols_in_portfolio):
            # Use percentile mapping for speed
            u = stats.norm.cdf(simulated_normals[:, i])
            perc_indices = np.floor(u * len(returns_matrix)).astype(int)
            perc_indices = np.clip(perc_indices, 0, len(returns_matrix) - 1)
            sorted_returns = np.sort(returns_matrix[:, i])
            simulated_returns[:, i] = sorted_returns[perc_indices]

        # Calculate portfolio returns
        portfolio_returns = simulated_returns @ weights_array

        # Calculate ES
        sorted_indices = np.argsort(portfolio_returns)
        var_index = int(n_simulations * (1 - confidence_level))
        tail_indices = sorted_indices[:var_index]

        # Calculate marginal contributions as average contribution in the tail
        mrc = {}
        for i, symbol in enumerate(symbols_in_portfolio):
            # Average contribution of this asset to losses in the tail
            tail_contribution = np.mean(simulated_returns[tail_indices, i])
            mrc[symbol] = -tail_contribution / (-np.mean(portfolio_returns[tail_indices]))

        return mrc

    except Exception as e:
        print(f"Error in ES calculation for {portfolio_name}: {e}")
        # Fallback to approximation if simulation fails
        mrc = {}
        for i, symbol in enumerate(symbols_in_portfolio):
            mrc[symbol] = marginal_contrib_approx[i]
        return mrc

# Define objective function for risk parity optimization
def risk_parity_objective(raw_weights, portfolio_name, symbols, confidence_level=0.95, n_simulations=500):
    """
    Objective function for risk parity optimization with efficiency improvements.
    We want to minimize the sum of squared differences between risk contributions.

    Args:
        raw_weights: Optimization variable (raw weights before normalization)
        portfolio_name: Name of the portfolio
        symbols: List of symbols in the portfolio
        confidence_level: Confidence level for ES
        n_simulations: Number of simulations

    Returns:
        Sum of squared differences between risk contributions
    """
    # Ensure positive weights and normalize to sum to 1
    weights = np.maximum(raw_weights, 1e-8)
    weights = weights / np.sum(weights)

    # Convert to dictionary format
    weights_dict = {symbols[i]: weights[i] for i in range(len(symbols))}

    # For portfolio with low simulation count, use variance-based approximation
    if n_simulations < 300:
        # Get returns data for all symbols
        returns_matrix = np.zeros((len(stock_returns[symbols[0]]), len(symbols)))
        for i, symbol in enumerate(symbols):
            returns_matrix[:, i] = stock_returns[symbol]

        # Calculate covariance matrix
        cov_matrix = np.cov(returns_matrix, rowvar=False)

        # Calculate portfolio variance
        portfolio_variance = weights @ cov_matrix @ weights

        # Calculate marginal risk contributions
        marginal_contributions = cov_matrix @ weights / np.sqrt(portfolio_variance)

        # Calculate risk contributions
        risk_contributions = weights * marginal_contributions

        # Calculate target risk contribution
        target_risk = np.sum(risk_contributions) / len(symbols)

        # Return sum of squared deviations
        return np.sum((risk_contributions - target_risk) ** 2)

    # For other portfolios, use ES-based risk contributions
    else:
        # Calculate marginal risk contributions
        mrc = calculate_marginal_risk_contributions(
            weights_dict,
            portfolio_name,
            confidence_level,
            n_simulations
        )

        # Calculate risk contributions
        rc = {symbol: weights_dict[symbol] * mrc[symbol] for symbol in symbols}
        total_rc = sum(rc.values())

        # Target: equal risk contribution from each asset
        target_rc = total_rc / len(symbols)

        # Sum of squared deviations from target
        return sum((rc[symbol] - target_rc) ** 2 for symbol in symbols)

# Function to create risk parity portfolio with multiple starts
def optimize_with_multiple_starts(portfolio_name, symbols, n_attempts=5, confidence_level=0.95, n_simulations=500):
    """
    Optimize risk parity portfolio with multiple random starts to avoid local minima.

    Args:
        portfolio_name: Name of the portfolio
        symbols: List of symbols in the portfolio
        n_attempts: Number of optimization attempts with different initial weights
        confidence_level: Confidence level for ES calculation
        n_simulations: Number of simulations for Monte Carlo

    Returns:
        Dictionary of optimized weights
    """
    print(f"Optimizing portfolio {portfolio_name} with {n_attempts} different starting points...")
    best_result = None
    best_objective = float('inf')
    n_assets = len(symbols)

    for i in range(n_attempts):
        # Generate different initial weights for each attempt
        np.random.seed(42 + i)

        # Use different strategies for different attempts
        if i == 0:
            # First attempt: equal weights
            initial_weights = np.ones(n_assets) / n_assets
        elif i == 1:
            # Second attempt: inverse variance weights (good for risk parity)
            # Get returns data
            returns_matrix = np.zeros((len(stock_returns[symbols[0]]), len(symbols)))
            for j, symbol in enumerate(symbols):
                returns_matrix[:, j] = stock_returns[symbol]

            # Calculate variances and inverse variance weights
            variances = np.var(returns_matrix, axis=0)
            inv_var = 1.0 / (variances + 1e-8)  # Add small constant to avoid division by zero
            initial_weights = inv_var / np.sum(inv_var)
        elif i == 2:
            # Third attempt: random weights with uniform concentration
            alpha = np.ones(n_assets)  # Equal concentration
            initial_weights = np.random.dirichlet(alpha)
        elif i == 3:
            # Fourth attempt: high concentration on low volatility assets
            returns_matrix = np.zeros((len(stock_returns[symbols[0]]), len(symbols)))
            for j, symbol in enumerate(symbols):
                returns_matrix[:, j] = stock_returns[symbol]

            volatilities = np.std(returns_matrix, axis=0)
            alpha = 1.0 / (volatilities + 1e-8)
            alpha = alpha / np.mean(alpha) * 5  # Scale to reasonable concentration
            initial_weights = np.random.dirichlet(alpha)
        else:
            # Fifth attempt: high concentration on high volatility assets (opposite of fourth)
            returns_matrix = np.zeros((len(stock_returns[symbols[0]]), len(symbols)))
            for j, symbol in enumerate(symbols):
                returns_matrix[:, j] = stock_returns[symbol]

            volatilities = np.std(returns_matrix, axis=0)
            alpha = volatilities + 1e-8
            alpha = alpha / np.mean(alpha) * 5  # Scale to reasonable concentration
            initial_weights = np.random.dirichlet(alpha)

        # Define constraints
        constraints = (
            {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}  # Sum of weights = 1
        )

        # Define bounds
        bounds = [(0.001, 1) for _ in range(n_assets)]  # Lower bound to avoid zero weights

        # Set optimizer options - use consistent settings for all portfolios
        optimizer_options = {
            'maxiter': 50,
            'ftol': 1e-4,
            'eps': 1e-3,
            'disp': True
        }

        if i > 0:
            # For all portfolios, use more aggressive settings on later attempts
            optimizer_options = {
                'maxiter': 100,  # Increased iterations
                'ftol': 1e-5,    # Tighter tolerance
                'eps': 5e-4,     # Smaller step size
                'disp': True
            }

        # Start timer
        start_time = time.time()

        # Optimize with SLSQP
        print(f"Attempt {i+1}/{n_attempts} for portfolio {portfolio_name}...")
        result = minimize(
            risk_parity_objective,
            initial_weights,
            args=(portfolio_name, symbols, confidence_level, n_simulations),
            method='SLSQP',
            bounds=bounds,
            constraints=constraints,
            options=optimizer_options
        )

        # Get optimized weights and normalize to ensure sum=1
        optimized_weights = result.x
        optimized_weights = optimized_weights / np.sum(optimized_weights)

        # Calculate objective function value
        objective = risk_parity_objective(
            optimized_weights,
            portfolio_name,
            symbols,
            confidence_level,
            n_simulations
        )

        print(f"Attempt {i+1} completed in {time.time() - start_time:.2f} seconds with objective value: {objective:.6f}")

        # Update if this is better than previous best
        if objective < best_objective:
            best_objective = objective
            best_result = {symbols[i]: optimized_weights[i] for i in range(n_assets)}
            print(f"New best result found in attempt {i+1} with objective value: {objective:.6f}")

    print(f"Best optimization result for portfolio {portfolio_name} with objective value: {best_objective:.6f}")
    return best_result

# Function to create risk parity portfolio (original)
def create_risk_parity_portfolio(portfolio_name, symbols, initial_weights=None, confidence_level=0.95, n_simulations=500, optimizer_options=None):
    """
    Create a risk parity portfolio for the given symbols with improved efficiency.

    Args:
        portfolio_name: Name of the portfolio
        symbols: List of symbols in the portfolio
        initial_weights: Initial weights to start optimization (equal if None)
        confidence_level: Confidence level for ES
        n_simulations: Number of simulations for Monte Carlo
        optimizer_options: Dictionary of options for the optimizer

    Returns:
        Dictionary of optimized weights
    """
    n_assets = len(symbols)

    # Start with equal weights if not provided
    if initial_weights is None:
        initial_weights = np.ones(n_assets) / n_assets

    # Add random perturbation if desired
    np.random.seed(42)
    perturbation = np.random.uniform(0.9, 1.1, size=n_assets)
    initial_weights = initial_weights * perturbation
    initial_weights = initial_weights / np.sum(initial_weights)

    # Define constraints
    constraints = (
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}  # Sum of weights = 1
    )

    # Define bounds
    bounds = [(0.001, 1) for _ in range(n_assets)]  # Lower bound to avoid zero weights

    # Set default optimizer options if none provided
    if optimizer_options is None:
        optimizer_options = {
            'maxiter': 150,
            'ftol': 1e-4,
            'eps': 1e-3,
            'disp': True
        }

    # Optimize with SLSQP
    print(f"Optimizing risk parity portfolio for {portfolio_name}...")
    result = minimize(
        risk_parity_objective,
        initial_weights,
        args=(portfolio_name, symbols, confidence_level, n_simulations),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options=optimizer_options
    )

    # Get optimized weights and normalize again to ensure sum=1
    optimized_weights = result.x
    optimized_weights = optimized_weights / np.sum(optimized_weights)

    # Convert to dictionary
    return {symbols[i]: optimized_weights[i] for i in range(n_assets)}

# Step 5: Create risk parity portfolios for each sub-portfolio
print("\n=== Creating Risk Parity Portfolios ===")
# Use portfolios from initial_portfolio
portfolios = initial_portfolio['Portfolio'].unique().tolist()
risk_parity_weights = {}

# Define parameters for each portfolio - give them all 500 simulations now
portfolio_params = {
    'A': {'n_sim': 5000, 'ftol': 1e-4, 'maxiter': 50, 'eps': 1e-3, 'multi_start': True},
    'B': {'n_sim': 5000, 'ftol': 1e-4, 'maxiter': 50, 'eps': 1e-3, 'multi_start': True},
    'C': {'n_sim': 5000, 'ftol': 1e-5, 'maxiter': 100, 'eps': 5e-4, 'multi_start': True}
}

# Process each portfolio
for portfolio in portfolios:
    # Get parameters for this portfolio
    params = portfolio_params.get(portfolio, {'n_sim': 500, 'ftol': 1e-4, 'maxiter': 50, 'eps': 1e-3, 'multi_start': False})

    # Get symbols for this portfolio
    portfolio_df = initial_portfolio[initial_portfolio['Portfolio'] == portfolio]
    portfolio_symbols = portfolio_df['Symbol'].unique().tolist()

    # Get initial weights
    portfolio_weights_dict = {}
    for symbol in portfolio_symbols:
        weight = portfolio_weights[portfolio][symbol]['weight']
        portfolio_weights_dict[symbol] = weight

    # Convert to numpy array
    initial_weights = np.array([portfolio_weights_dict[symbol] for symbol in portfolio_symbols])

    # For portfolio C, use multi-start optimization
    if params['multi_start']:
        print(f"Using multi-start optimization for portfolio {portfolio} with {params['n_sim']} simulations")
        risk_parity_weights[portfolio] = optimize_with_multiple_starts(
            portfolio,
            portfolio_symbols,
            n_attempts=3,  # Try 3 different starting points
            confidence_level=0.95,
            n_simulations=params['n_sim']
        )
    else:
        # Define optimizer options
        optimizer_options = {
            'maxiter': params['maxiter'],
            'ftol': params['ftol'],
            'eps': params['eps'],
            'disp': True
        }

        # Create risk parity portfolio
        risk_parity_weights[portfolio] = create_risk_parity_portfolio(
            portfolio,
            portfolio_symbols,
            initial_weights=initial_weights,
            confidence_level=0.95,
            n_simulations=params['n_sim'],
            optimizer_options=optimizer_options
        )

# Step 6: Calculate VaR and ES for risk parity portfolios
print("\n=== Risk Metrics for Risk Parity Portfolios ===")
risk_parity_var_es = {}

for portfolio in portfolios:
    # Calculate VaR and ES for risk parity portfolio
    var_gc, es_gc = calculate_var_es(
        portfolio,
        risk_parity_weights[portfolio],
        confidence_level=0.95,
        method="GaussianCopula",
        n_simulations=1000
    )

    risk_parity_var_es[portfolio] = {
        'VaR': var_gc,
        'ES': es_gc
    }

# Step 7: Compare original vs risk parity portfolios
print("\nComparison of Original vs Risk Parity Portfolios:")
print("=" * 100)
print(f"{'Portfolio':<10} {'Original VaR':<15} {'Original ES':<15} {'RP VaR':<15} {'RP ES':<15} {'VaR Change %':<15} {'ES Change %':<15}")
print("-" * 100)

for portfolio in portfolios:
    # Check if we have risk metrics for the original portfolio
    if portfolio not in var_es_results:
        print(f"Calculating original risk metrics for portfolio {portfolio}...")
        # Calculate VaR and ES for the original portfolio
        weights = {}
        for symbol in portfolio_weights[portfolio]:
            weights[symbol] = portfolio_weights[portfolio][symbol]['weight']

        var_gc, es_gc = calculate_var_es(
            portfolio, weights,
            confidence_level=0.95,
            method="GaussianCopula",
            n_simulations=10000
        )

        # Store results
        var_es_results[portfolio] = {
            'GaussianCopula': {'VaR': var_gc, 'ES': es_gc}
        }

    # Now get results for comparison
    orig_var = var_es_results[portfolio]['GaussianCopula']['VaR']
    orig_es = var_es_results[portfolio]['GaussianCopula']['ES']
    rp_var = risk_parity_var_es[portfolio]['VaR']
    rp_es = risk_parity_var_es[portfolio]['ES']

    # Calculate percentage changes
    var_change = (rp_var - orig_var) / orig_var * 100
    es_change = (rp_es - orig_es) / orig_es * 100

    print(f"{portfolio:<10} {orig_var*100:<15.4f}% {orig_es*100:<15.4f}% {rp_var*100:<15.4f}% {rp_es*100:<15.4f}% {var_change:<15.2f}% {es_change:<15.2f}%")

# Step 8: Print risk parity portfolio weights
print("\nRisk Parity Portfolio Weights:")
print("=" * 100)
for portfolio in portfolios:
    print(f"\nPortfolio {portfolio}:")
    print("-" * 50)
    print(f"{'Symbol':<8} {'Original Weight':<20} {'Risk Parity Weight':<20} {'Change':<15}")
    print("-" * 65)

    # Get original weights
    for symbol in risk_parity_weights[portfolio]:
        orig_weight = portfolio_weights[portfolio][symbol]['weight']
        rp_weight = risk_parity_weights[portfolio][symbol]
        weight_change = rp_weight - orig_weight

        print(f"{symbol:<8} {orig_weight*100:<19.2f}% {rp_weight*100:<19.2f}% {weight_change*100:+<14.2f}%")

# Step 9: Calculate and print risk contributions
print("\nRisk Contributions Analysis:")
print("=" * 100)

for portfolio in portfolios:
    print(f"\nPortfolio {portfolio} - Risk Contributions:")
    print("-" * 70)
    print(f"{'Symbol':<8} {'Risk Parity Weight':<20} {'Marginal Contribution':<25} {'% of Total Risk':<20}")
    print("-" * 70)

    # Calculate marginal risk contributions using the portfolio's simulation count
    mrc = calculate_marginal_risk_contributions(
        risk_parity_weights[portfolio],
        portfolio,
        confidence_level=0.95,
        n_simulations=portfolio_params[portfolio]['n_sim']
    )

    # Calculate risk contributions
    rc = {symbol: risk_parity_weights[portfolio][symbol] * mrc[symbol] for symbol in risk_parity_weights[portfolio]}
    total_rc = sum(rc.values())

    # Print results
    for symbol in risk_parity_weights[portfolio]:
        rc_pct = rc[symbol] / total_rc * 100

        print(f"{symbol:<8} {risk_parity_weights[portfolio][symbol]*100:<19.2f}% {mrc[symbol]:<24.4f} {rc_pct:<19.2f}%")

    # Verify risk parity: all assets should have approximately equal risk contribution percentages
    avg_rc_pct = 100 / len(risk_parity_weights[portfolio])
    rc_pcts = [rc[symbol] / total_rc * 100 for symbol in risk_parity_weights[portfolio]]
    max_deviation = max([abs(pct - avg_rc_pct) for pct in rc_pcts])

    print(f"\nTarget risk contribution per asset: {avg_rc_pct:.2f}%")
    print(f"Maximum deviation from target: {max_deviation:.2f}%")

    # Much more relaxed condition for risk parity achievement for A and B
    if max_deviation <= 2.0:
        print("✓ Risk parity achieved (all assets contribute approximately equally to risk)")
    else:
        print("⚠ Risk parity not fully achieved - may need more optimization iterations")




=== Creating Risk Parity Portfolios ===
Using multi-start optimization for portfolio A with 5000 simulations
Optimizing portfolio A with 3 different starting points...
Attempt 1/3 for portfolio A...
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00034125399500475586
            Iterations: 6
            Function evaluations: 211
            Gradient evaluations: 6
Attempt 1 completed in 2.34 seconds with objective value: 0.000341
New best result found in attempt 1 with objective value: 0.000341
Attempt 2/3 for portfolio A...
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.161107646556641e-05
            Iterations: 12
            Function evaluations: 423
            Gradient evaluations: 12
Attempt 2 completed in 4.78 seconds with objective value: 0.000022
New best result found in attempt 2 with objective value: 0.000022
Attempt 3/3 for portfolio A...
Optimization terminated successfully    (Exit

In [11]:
import problem1
import problem2

# Step 10: Calculate return attribution for risk parity portfolios

# First, get all the necessary data from problem1
print("\nProblem 1:")
capm_results = problem1.run_capm_analysis()
print("\nProblem 2:")
# Also get results from problem2
sharpe_results = problem2.run_optimal_sharpe_analysis()
print("\nProblem 5:")


# Read necessary data files
daily_prices = pd.read_csv('../Projects/Final Project/DailyPrices.csv')
rf_data = pd.read_csv('../Projects/Final Project/rf.csv')

# Data preprocessing
daily_prices['Date'] = pd.to_datetime(daily_prices['Date'])
daily_prices.set_index('Date', inplace=True)

rf_data['Date'] = pd.to_datetime(rf_data['Date'])
rf_data.set_index('Date', inplace=True)

# Find the end of 2023
end_of_2023 = daily_prices[daily_prices.index.year == 2023].index.max()
last_date = daily_prices.index.max()

# Extract necessary prices
end_of_2023_prices = daily_prices.loc[end_of_2023]
last_day_prices = daily_prices.loc[last_date]

# Get CAPM parameters
capm_params = capm_results['capm_params']

# Calculate stock simple returns (already done in problem1 and problem2, but for completeness)
stock_simple_returns = {}
for symbol in daily_prices.columns:
    if symbol in end_of_2023_prices and symbol in last_day_prices:
        initial_price = end_of_2023_prices[symbol]
        final_price = last_day_prices[symbol]
        if not np.isnan(initial_price) and not np.isnan(final_price) and initial_price > 0:
            stock_simple_returns[symbol] = (final_price - initial_price) / initial_price
        else:
            stock_simple_returns[symbol] = np.nan

# Market return (SPY)
spy_return = stock_simple_returns['SPY']

# Get risk-free return
test_rf = rf_data.loc[rf_data.index > end_of_2023].squeeze()
test_rf_return = (1 + test_rf).prod() - 1

# Create a structure similar to initial_portfolio but with risk parity weights
rp_holdings = []
for portfolio_name in risk_parity_weights:
    for symbol, weight in risk_parity_weights[portfolio_name].items():
        # Calculate new holdings based on portfolio value
        portfolio_value = capm_results['portfolio_values'][portfolio_name]['initial_value']
        symbol_price = end_of_2023_prices[symbol]
        holding = (portfolio_value * weight) / symbol_price
        rp_holdings.append({
            'Portfolio': portfolio_name,
            'Symbol': symbol,
            'Holding': holding
        })

rp_portfolio_df = pd.DataFrame(rp_holdings)

# Calculate risk parity portfolio values and returns
rp_portfolio_values = {}

for portfolio_name in risk_parity_weights:
    portfolio_stocks = rp_portfolio_df[rp_portfolio_df['Portfolio'] == portfolio_name]

    initial_stock_values = {}
    final_stock_values = {}
    total_initial_value = 0
    total_final_value = 0

    # Calculate portfolio beta
    portfolio_beta = 0

    for _, row in portfolio_stocks.iterrows():
        symbol = row['Symbol']
        holding = row['Holding']

        if (symbol in end_of_2023_prices and not np.isnan(end_of_2023_prices[symbol]) and
            symbol in last_day_prices and not np.isnan(last_day_prices[symbol])):

            initial_value = holding * end_of_2023_prices[symbol]
            final_value = holding * last_day_prices[symbol]

            initial_stock_values[symbol] = initial_value
            final_stock_values[symbol] = final_value

            total_initial_value += initial_value
            total_final_value += final_value

    # Recalculate portfolio beta
    portfolio_beta = 0
    for symbol, initial_value in initial_stock_values.items():
        if symbol in capm_params:
            stock_beta = capm_params[symbol]['beta']
        else:
            stock_beta = 0

        portfolio_beta += (initial_value / total_initial_value) * stock_beta if total_initial_value > 0 else 0

    # Calculate simple return
    simple_return = (total_final_value - total_initial_value) / total_initial_value if total_initial_value > 0 else 0

    rp_portfolio_values[portfolio_name] = {
        'initial_value': total_initial_value,
        'final_value': total_final_value,
        'simple_return': simple_return,
        'initial_stock_values': initial_stock_values,
        'final_stock_values': final_stock_values,
        'portfolio_beta': portfolio_beta
    }

# Calculate return attribution for risk parity portfolios
rp_portfolio_attributions = {}

for portfolio_name, portfolio_values_data in rp_portfolio_values.items():
    total_return = portfolio_values_data['simple_return']
    portfolio_beta = portfolio_values_data['portfolio_beta']

    # Calculate return attribution
    systematic_return = portfolio_beta * spy_return
    idiosyncratic_return = total_return - systematic_return

    # Store attribution results
    rp_portfolio_attributions[portfolio_name] = {
        'total_return': total_return,
        'rf_return': test_rf_return,
        'systematic_return': systematic_return,
        'idiosyncratic_return': idiosyncratic_return,
        'total_excess_return': total_return - test_rf_return,
        'portfolio_beta': portfolio_beta
    }

# Calculate total risk parity portfolio attribution
total_rp_initial_value = sum(pv['initial_value'] for pv in rp_portfolio_values.values())
total_rp_final_value = sum(pv['final_value'] for pv in rp_portfolio_values.values())

# Calculate total simple return
total_rp_simple_return = (total_rp_final_value - total_rp_initial_value) / total_rp_initial_value if total_rp_initial_value > 0 else 0

# Calculate total portfolio beta
total_rp_portfolio_beta = 0
for portfolio_name, portfolio_data in rp_portfolio_values.items():
    weight = portfolio_data['initial_value'] / total_rp_initial_value
    total_rp_portfolio_beta += weight * portfolio_data['portfolio_beta']

# Calculate total return attribution
total_rp_systematic_return = total_rp_portfolio_beta * spy_return
total_rp_idiosyncratic_return = total_rp_simple_return - total_rp_systematic_return

total_rp_portfolio_attribution = {
    'total_return': total_rp_simple_return,
    'rf_return': test_rf_return,
    'systematic_return': total_rp_systematic_return,
    'idiosyncratic_return': total_rp_idiosyncratic_return,
    'total_excess_return': total_rp_simple_return - test_rf_return,
    'portfolio_beta': total_rp_portfolio_beta,
    'weights': {}
}

def calculate_volatility_attribution(portfolio_weights, portfolio_name, stock_returns, market_symbol='SPY'):
    """
    计算投资组合的波动率归因

    Args:
        portfolio_weights: 投资组合权重字典
        portfolio_name: 投资组合名称
        stock_returns: 股票收益率数据
        market_symbol: 市场指数符号，默认为SPY

    Returns:
        字典，包含系统性风险、特异性风险和总风险
    """
    # 收集投资组合中股票的回报率
    portfolio_symbols = list(portfolio_weights.keys())
    returns_data = {symbol: stock_returns[symbol] for symbol in portfolio_symbols if symbol in stock_returns}

    # 计算投资组合回报率
    portfolio_returns = np.zeros(len(stock_returns[market_symbol]))
    for symbol, weight in portfolio_weights.items():
        if symbol in stock_returns:
            portfolio_returns += weight * stock_returns[symbol]

    # 计算投资组合总波动率
    portfolio_volatility = np.std(portfolio_returns)

    # 计算市场回报率与投资组合回报率的相关性
    market_returns = stock_returns[market_symbol]

    # 运行单因子回归 R_p = α + β * R_m + ε
    X = market_returns.reshape(-1, 1)
    y = portfolio_returns

    # 添加常数项
    X_with_const = np.column_stack([np.ones(X.shape[0]), X])

    # 最小二乘法计算系数
    beta_alpha = np.linalg.lstsq(X_with_const, y, rcond=None)[0]
    alpha = beta_alpha[0]
    beta = beta_alpha[1]

    # 计算拟合值和残差
    fitted_returns = alpha + beta * market_returns
    residuals = portfolio_returns - fitted_returns

    # 计算系统性风险（β * σ_m）
    market_volatility = np.std(market_returns)
    systematic_risk = beta * market_volatility

    # 计算特异性风险（残差波动率）
    idiosyncratic_risk = np.std(residuals)

    # 验证：总风险²应约等于系统性风险²+特异性风险²
    # portfolio_variance = systematic_risk**2 + idiosyncratic_risk**2

    return {
        'spy': systematic_risk,
        'alpha': idiosyncratic_risk,
        'portfolio': portfolio_volatility
    }

# Calculate weights of each portfolio in the total
for portfolio_name, portfolio_data in rp_portfolio_values.items():
    weight = portfolio_data['initial_value'] / total_rp_initial_value
    total_rp_portfolio_attribution['weights'][portfolio_name] = weight

# Calculate volatility attribution for risk parity portfolios (simplified)
rp_vol_attribution = {}

# 处理每个子投资组合
for portfolio_name in portfolios:
    rp_vol_attribution[portfolio_name] = calculate_volatility_attribution(
        risk_parity_weights[portfolio_name],
        portfolio_name,
        stock_returns
    )

# 计算总体投资组合的风险归因
# 首先合并所有风险平价投资组合的权重
total_rp_weights = {}
for portfolio_name, weights in risk_parity_weights.items():
    portfolio_weight = total_rp_portfolio_attribution['weights'][portfolio_name]
    for symbol, weight in weights.items():
        if symbol in total_rp_weights:
            total_rp_weights[symbol] += weight * portfolio_weight
        else:
            total_rp_weights[symbol] = weight * portfolio_weight

# 计算总体投资组合的风险归因
rp_vol_attribution['Total'] = calculate_volatility_attribution(
    total_rp_weights,
    'Total',
    stock_returns
)

# Print attribution results using format from problem1
print("\n")
problem1.print_attribution_results(rp_portfolio_attributions, total_rp_portfolio_attribution,
                             stock_simple_returns, rp_vol_attribution)

# Step 11: Compare all three strategies
print("\n=== Comparison of All Three Portfolio Strategies ===")

# Get original portfolio attribution
original_portfolio_attributions = capm_results['portfolio_attributions']
original_total_attribution = capm_results['total_portfolio_attribution']

# Get optimal Sharpe ratio portfolio attribution
sharpe_portfolio_attributions = sharpe_results['optimal_portfolio_attributions']
sharpe_total_attribution = sharpe_results['optimal_total_portfolio_attribution']

# Create comparison table
print("\nTotal Portfolio Comparison:")
print("=" * 100)
print(f"{'Metric':<20} {'Original':<15} {'Optimal Sharpe':<15} {'Risk Parity':<15} {'OS vs Orig':<15} {'RP vs Orig':<15}")
print("-" * 100)

# Total return
orig_return = original_total_attribution['total_return']
sharpe_return = sharpe_total_attribution['total_return']
rp_return = total_rp_portfolio_attribution['total_return']
print(f"{'Total Return':<20} {orig_return*100:14.2f}% {sharpe_return*100:14.2f}% {rp_return*100:14.2f}% {(sharpe_return-orig_return)*100:14.2f}% {(rp_return-orig_return)*100:14.2f}%")

# Systematic return
orig_sys = original_total_attribution['systematic_return']
sharpe_sys = sharpe_total_attribution['systematic_return']
rp_sys = total_rp_portfolio_attribution['systematic_return']
print(f"{'Systematic Return':<20} {orig_sys*100:14.2f}% {sharpe_sys*100:14.2f}% {rp_sys*100:14.2f}% {(sharpe_sys-orig_sys)*100:14.2f}% {(rp_sys-orig_sys)*100:14.2f}%")

# Idiosyncratic return
orig_idio = original_total_attribution['idiosyncratic_return']
sharpe_idio = sharpe_total_attribution['idiosyncratic_return']
rp_idio = total_rp_portfolio_attribution['idiosyncratic_return']
print(f"{'Idiosyncratic Return':<20} {orig_idio*100:14.2f}% {sharpe_idio*100:14.2f}% {rp_idio*100:14.2f}% {(sharpe_idio-orig_idio)*100:14.2f}% {(rp_idio-orig_idio)*100:14.2f}%")

# Beta
orig_beta = original_total_attribution['portfolio_beta']
sharpe_beta = sharpe_total_attribution['portfolio_beta']
rp_beta = total_rp_portfolio_attribution['portfolio_beta']
print(f"{'Portfolio Beta':<20} {orig_beta:14.2f} {sharpe_beta:14.2f} {rp_beta:14.2f} {(sharpe_beta-orig_beta):14.2f} {(rp_beta-orig_beta):14.2f}")

# Compare each sub-portfolio
for portfolio_name in portfolios:
    print(f"\nComparison for Portfolio {portfolio_name}:")
    print("=" * 100)
    print(f"{'Metric':<20} {'Original':<15} {'Optimal Sharpe':<15} {'Risk Parity':<15} {'OS vs Orig':<15} {'RP vs Orig':<15}")
    print("-" * 100)

    orig_return = original_portfolio_attributions[portfolio_name]['total_return']
    sharpe_return = sharpe_portfolio_attributions[portfolio_name]['total_return']
    rp_return = rp_portfolio_attributions[portfolio_name]['total_return']
    print(f"{'Total Return':<20} {orig_return*100:14.2f}% {sharpe_return*100:14.2f}% {rp_return*100:14.2f}% {(sharpe_return-orig_return)*100:14.2f}% {(rp_return-orig_return)*100:14.2f}%")

    orig_sys = original_portfolio_attributions[portfolio_name]['systematic_return']
    sharpe_sys = sharpe_portfolio_attributions[portfolio_name]['systematic_return']
    rp_sys = rp_portfolio_attributions[portfolio_name]['systematic_return']
    print(f"{'Systematic Return':<20} {orig_sys*100:14.2f}% {sharpe_sys*100:14.2f}% {rp_sys*100:14.2f}% {(sharpe_sys-orig_sys)*100:14.2f}% {(rp_sys-orig_sys)*100:14.2f}%")

    orig_idio = original_portfolio_attributions[portfolio_name]['idiosyncratic_return']
    sharpe_idio = sharpe_portfolio_attributions[portfolio_name]['idiosyncratic_return']
    rp_idio = rp_portfolio_attributions[portfolio_name]['idiosyncratic_return']
    print(f"{'Idiosyncratic Return':<20} {orig_idio*100:14.2f}% {sharpe_idio*100:14.2f}% {rp_idio*100:14.2f}% {(sharpe_idio-orig_idio)*100:14.2f}% {(rp_idio-orig_idio)*100:14.2f}%")

    orig_beta = original_portfolio_attributions[portfolio_name]['portfolio_beta']
    sharpe_beta = sharpe_portfolio_attributions[portfolio_name]['portfolio_beta']
    rp_beta = rp_portfolio_attributions[portfolio_name]['portfolio_beta']
    print(f"{'Portfolio Beta':<20} {orig_beta:14.2f} {sharpe_beta:14.2f} {rp_beta:14.2f} {(sharpe_beta-orig_beta):14.2f} {(rp_beta-orig_beta):14.2f}")

    # Add risk metrics
    if portfolio_name in risk_parity_var_es:
        orig_var = var_es_results[portfolio_name]['GaussianCopula']['VaR'] * 100
        orig_es = var_es_results[portfolio_name]['GaussianCopula']['ES'] * 100
        rp_var = risk_parity_var_es[portfolio_name]['VaR'] * 100
        rp_es = risk_parity_var_es[portfolio_name]['ES'] * 100

        # For Sharpe ratio portfolios, we don't have these metrics, so we'll leave them blank
        print(f"{'VaR (95%)':<20} {orig_var:14.2f}% {'-':>14} {rp_var:14.2f}% {'-':>14} {(rp_var-orig_var):14.2f}%")
        print(f"{'ES (95%)':<20} {orig_es:14.2f}% {'-':>14} {rp_es:14.2f}% {'-':>14} {(rp_es-orig_es):14.2f}%")

# Step 12: Print a summary of findings
print("\n=== Summary of Portfolio Strategy Comparison ===")
print("\nKey Findings:")

# Compare total returns
strategies = ["Original", "Optimal Sharpe", "Risk Parity"]
returns = [orig_return*100, sharpe_return*100, rp_return*100]
best_return_idx = returns.index(max(returns))
print(f"1. Best overall return: {strategies[best_return_idx]} portfolio with {max(returns):.2f}%")

# Compare systematic risk (beta)
betas = [orig_beta, sharpe_beta, rp_beta]
lowest_beta_idx = betas.index(min(betas))
print(f"2. Lowest systematic risk (beta): {strategies[lowest_beta_idx]} portfolio with beta of {min(betas):.2f}")

# Compare risk metrics
if 'A' in risk_parity_var_es:
    print(f"3. Risk Parity approach reduced risk compared to original portfolio:")
    for portfolio_name in portfolios:
        var_change = (risk_parity_var_es[portfolio_name]['VaR'] - var_es_results[portfolio_name]['GaussianCopula']['VaR']) / var_es_results[portfolio_name]['GaussianCopula']['VaR'] * 100
        es_change = (risk_parity_var_es[portfolio_name]['ES'] - var_es_results[portfolio_name]['GaussianCopula']['ES']) / var_es_results[portfolio_name]['GaussianCopula']['ES'] * 100

        print(f"   - Portfolio {portfolio_name}: VaR changed by {var_change:.2f}%, ES changed by {es_change:.2f}%")

# Compare idiosyncratic returns
idio_returns = [orig_idio*100, sharpe_idio*100, rp_idio*100]
best_idio_idx = idio_returns.index(max(idio_returns))
print(f"4. Best idiosyncratic return: {strategies[best_idio_idx]} portfolio with {max(idio_returns):.2f}%")



Problem 1:
开始执行CAPM投资组合风险与收益归因分析...
训练集结束日期: 2023-12-29
训练集天数: 250
测试集天数: 254

部分股票的CAPM参数:
AAPL: Beta=1.10, Alpha=0.0008, R²=0.53
MSFT: Beta=1.17, Alpha=0.0009, R²=0.37
AMZN: Beta=1.53, Alpha=0.0011, R²=0.37
GOOGL: Beta=1.38, Alpha=0.0007, R²=0.35
META: Beta=1.77, Alpha=0.0029, R²=0.34

各投资组合价值和简单回报率:
A: 初始值=$295444.61, 最终值=$335814.67, 回报率=13.66%, Beta=0.97
B: 初始值=$280904.48, 最终值=$338075.80, 回报率=20.35%, Beta=0.92
C: 初始值=$267591.44, 最终值=$342830.78, 回报率=28.12%, Beta=0.97

主要股票的简单回报率:
SPY: 26.14%
AAPL: 27.02%
MSFT: 13.20%
AMZN: 47.55%
GOOGL: 37.79%

无风险回报率: 5.27%


# Total Portfolio Attribution
# ----------------------------------------------------------------------
#  Row | Value                           SPY         Alpha     Portfolio
# ----------------------------------------------------------------------
#  1   | Total Return               0.261373     -0.056642      0.204731
#  2   | Return Attribution         0.249311     -0.044580      0.204731
#  3   | Vol Attribution          